In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv

In [ ]:
driver=webdriver.Chrome()
wait = WebDriverWait(driver, 10)  

In [ ]:
import time
driver.get(url="https://www.thehindu.com/search/#gsc.tab=0&gsc.q=Indian%20Economy&gsc.sort=date")
wait = WebDriverWait(driver, 12) 

In [ ]:
articles_dict = {}
k = 0
iter_flag = True
pageNo = 0

while(iter_flag):
    nextPageUrl=f'https://www.thehindu.com/search/#gsc.tab=0&gsc.q=Indian%20Economy&gsc.sort=date&gsc.page={pageNo}'
    print(nextPageUrl)
    pageNo=pageNo+1
    driver.get(url=nextPageUrl)
    
    for j in range(1, 11):
        
        try:
            articles = wait.until(EC.visibility_of_element_located((By.XPATH, f'/html/body/section[2]/div/div[2]/div[2]/div/div/div/div/div[5]/div[2]/div/div/div[1]/div[{j}]/div/div[1]/div/a')))
            print(articles.text)
            # publish_details = driver.find_element(By.XPATH, )
            details = {
                'id': k,
                'publisher': "The Hindu",
                'title': articles.text,
                'link': articles.get_attribute('href'),
                'publish_details': '',
                'content': ""
            }
            # print(details)
            articles_dict[f'{k}'] = details
            k=k+1
                        
        except Exception as e:
            print('Exception encountered at : ', k)
            driver.refresh()
            
            articles=wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/section[2]/div/div[2]/div[2]/div/div/div/div/div[5]/div[2]/div/div/div[1]/div[{j}]/div/div[1]/div/a')))
            details = {
                'id': k,
                'publisher': "The Hindu",
                'title': '',
                'link': '',
                'publish_details': '',
                'content': ""
            }
            # print(details)
            articles_dict[f'{k}'] = details
            k=k+1
            
 
    if (k >= 501):
        iter_flag = False
    else:
        print(pageNo)      

print(articles_dict)


In [ ]:
from newspaper import Article

def extract_content(url):
    # Create an Article object
    article = Article(url)

    # Download and parse the article
    article.download()
    article.parse()
    
    return article.text

for keys in articles_dict.keys():
    link=articles_dict[keys]['link']
    # driver.get(url=link)    
    # date=driver.find_element(By.XPATH,"/html/body/section[1]/section/div[3]/div/div[2]/div[1]").text
    # content= driver.find_element(By.XPATH,'/html/body/section[1]/section/div[3]/div').text
    content=extract_content(link)
    
    articles_dict[keys]['content']= content

In [ ]:
for keys in articles_dict.keys():
    link=articles_dict[keys]['link']
    print(link)
    driver.get(link)
    try:
        content=driver.find_element(By.XPATH,'/html/body/section[2]/div/div/div[1]/div[3]/div[4]').text
    except Exception as e:
        try:
            content=driver.find_element(By.XPATH,'/html/body/section[2]/div/div/div[1]/div[3]/div[3]').text
        except Exception as f:
            content=''
    finally:
        articles_dict[keys]['content']=content

In [ ]:
print(articles_dict)

In [ ]:
# Flatten the nested structure
flat_articles_list = []
for article_id, article_data in articles_dict.items():
    flat_article = {'id': article_data['id'], 'publisher': article_data['publisher'], 'title': article_data['title'], 'link': article_data['link'], 'publish_details': article_data['publish_details'], 'content': article_data['content']}
    flat_articles_list.append(flat_article)

# Specify the CSV file path
csv_file_path = 'theHindu1.csv'

# Write the flattened data to a CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['id', 'publisher', 'title', 'link', 'publish_details', 'content']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write the header
    writer.writeheader()
    
    # Write the data
    writer.writerows(flat_articles_list)

print(f"The data has been written to '{csv_file_path}'.")

In [2]:
import pandas as pd
from newspaper import Article

# Load the CSV file into a DataFrame
df = pd.read_csv('theHindu1.csv')

# Function to extract content from a given link using newspaper3k
def extract_content(link):
    try:
        article = Article(link)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"Error extracting content from {link}: {str(e)}")
        return None

# Apply the extract_content function to update the 'content' column
df['content'] = df['link'].apply(extract_content)

# Save the updated DataFrame to a new CSV file
df.to_csv('theHindu_withContent.csv', index=False)

print("Content extraction and update completed. Updated data saved to 'updated_data.csv'.")


Error extracting content from nan: 'float' object has no attribute 'decode'
Error extracting content from nan: 'float' object has no attribute 'decode'
Error extracting content from nan: 'float' object has no attribute 'decode'
Error extracting content from nan: 'float' object has no attribute 'decode'
Content extraction and update completed. Updated data saved to 'updated_data.csv'.
